# Now Insurance Loss Runs #
## automatic extraction of relevant information in loss reports from pdf files ##

In [ ]:
## load depedencies 
import numpy as np # math library
import cv2 # visual computational 
import matplotlib.pyplot as plt # vizualization
import os # directory management
from configobj import ConfigObj # load rules
import lossrun

In [ ]:
## Read raw txt info in pdf report

PATH_txt = os.path.join('.','data','txt','')
PATH_image = os.path.join('.','data','images','')

# pdf raw file no ext 
pdf_file = 'AHC-0000343 loss runs PIO-YES0'
#  Read text file 
txt_file = PATH_txt + pdf_file + '.txt'
#image file
image_file = PATH_image + pdf_file + '.jpg'

# Read as dict format txt file
txt_dict = lossrun.read_dict(txt_file)

# load configuration file
template_rules = ConfigObj('config_file.ino')



## Sedgwick ##

In [ ]:
pdf_file = '12 - 13 loss runs - Focus Nursing CHG770310301 11-1-12-13_YES0'
#  Read text file 
txt_file = PATH_txt + pdf_file + '.txt'
#image file
image_file = PATH_image + pdf_file + '.jpg'

# Read as dict format txt file
txt_dict = lossrun.read_dict(txt_file)

template_rules = ConfigObj('config_file.ino')

j = lossrun.search_rules(txt_dict,template_rules)
_image_c = cv2.imread(image_file)
for i in range(len(j)):
    cv2.circle(_image_c,(j[i][0],j[i][1]),10,(0,0,255),-1)
plt.figure(figsize=(30,25))
plt.imshow(_image_c)

In [ ]:
pdf_file = 'BMS loss runs 11-12-Indian Harbor Sed-YES0'
#  Read text file 
txt_file = PATH_txt + pdf_file + '.txt'
#image file
image_file = PATH_image + pdf_file + '.jpg'

# Read as dict format txt file
txt_dict = lossrun.read_dict(txt_file)

template_rules = ConfigObj('config_file.ino')

j = lossrun.search_rules(txt_dict,template_rules)
_image_c = cv2.imread(image_file)
for i in range(len(j)):
    cv2.circle(_image_c,(j[i][0],j[i][1]),10,(0,0,255),-1)
plt.figure(figsize=(30,25))
plt.imshow(_image_c)

In [ ]:
pdf_file = 'Yr 15-16 thru 16-17 Rockhill loss runs valued 10-30-2018 SED-YES1'
#  Read text file 
txt_file = PATH_txt + pdf_file + '.txt'
#image file
image_file = PATH_image + pdf_file + '.jpg'

# Read as dict format txt file
txt_dict = lossrun.read_dict(txt_file)

template_rules = ConfigObj('config_file.ino')

j = lossrun.search_rules(txt_dict,template_rules)
_image_c = cv2.imread(image_file)
for i in range(len(j)):
    cv2.circle(_image_c,(j[i][0],j[i][1]),10,(0,0,255),-1)
plt.figure(figsize=(30,25))
plt.imshow(_image_c)

## Pioner ##

In [ ]:
pdf_file = 'Acts Home Pioneer loss runs PIO-NO1'
#  Read text file 
txt_file = PATH_txt + pdf_file + '.txt'
#image file
image_file = PATH_image + pdf_file + '.jpg'

# Read as dict format txt file
txt_dict = lossrun.read_dict(txt_file)

template_rules = ConfigObj('config_file.ino')

j = lossrun.search_rules(txt_dict,template_rules)
_image_c = cv2.imread(image_file)
for i in range(len(j)):
    cv2.circle(_image_c,(j[i][0],j[i][1]),10,(0,0,255),-1)
plt.figure(figsize=(30,25))
plt.imshow(_image_c)

In [ ]:
pdf_file = 'A-MMED 2017 Pioneer loss runs PIO-YES0'
#  Read text file 
txt_file = PATH_txt + pdf_file + '.txt'
#image file
image_file = PATH_image + pdf_file + '.jpg'

# Read as dict format txt file
txt_dict = lossrun.read_dict(txt_file)

template_rules = ConfigObj('config_file.ino')

j = lossrun.search_rules(txt_dict,template_rules)
_image_c = cv2.imread(image_file)
for i in range(len(j)):
    cv2.circle(_image_c,(j[i][0],j[i][1]),10,(0,0,255),-1)
plt.figure(figsize=(30,25))
plt.imshow(_image_c)

In [ ]:
pdf_file = 'Acts Home care loss runs PIO-NO1'
#  Read text file 
txt_file = PATH_txt + pdf_file + '.txt'
#image file
image_file = PATH_image + pdf_file + '.jpg'

# Read as dict format txt file
txt_dict = lossrun.read_dict(txt_file)

template_rules = ConfigObj('config_file.ino')

j = lossrun.search_rules(txt_dict,template_rules)
_image_c = cv2.imread(image_file)
for i in range(len(j)):
    cv2.circle(_image_c,(j[i][0],j[i][1]),10,(0,0,255),-1)
plt.figure(figsize=(30,25))
plt.imshow(_image_c)

## VGM ##

In [ ]:
pdf_file = 'Alternative Home Health Care of Broward  -   VGM19722077218-4   -lossrun VGM-YES0'
#  Read text file 
txt_file = PATH_txt + pdf_file + '.txt'
#image file
image_file = PATH_image + pdf_file + '.jpg'

# Read as dict format txt file
txt_dict = lossrun.read_dict(txt_file)

template_rules = ConfigObj('config_file.ino')

j = lossrun.search_rules(txt_dict,template_rules)
_image_c = cv2.imread(image_file)
for i in range(len(j)):
    cv2.circle(_image_c,(j[i][0],j[i][1]),10,(0,0,255),-1)
plt.figure(figsize=(30,25))
plt.imshow(_image_c)

In [ ]:
pdf_file = 'CCCZ Enterprises    -    VGM19722063918-4 -lossrun VGM-NO0'
#  Read text file 
txt_file = PATH_txt + pdf_file + '.txt'
#image file
image_file = PATH_image + pdf_file + '.jpg'

# Read as dict format txt file
txt_dict = lossrun.read_dict(txt_file)

template_rules = ConfigObj('config_file.ino')

j = lossrun.search_rules(txt_dict,template_rules)
_image_c = cv2.imread(image_file)
for i in range(len(j)):
    cv2.circle(_image_c,(j[i][0],j[i][1]),10,(0,0,255),-1)
plt.figure(figsize=(30,25))
plt.imshow(_image_c)

In [ ]:
pdf_file = 'Hamilton Park Dialysis OPCO   -   VGM16712077718-4         -lossrun VGM-NO0'
#  Read text file 
txt_file = PATH_txt + pdf_file + '.txt'
#image file
image_file = PATH_image + pdf_file + '.jpg'

# Read as dict format txt file
txt_dict = lossrun.read_dict(txt_file)

template_rules = ConfigObj('config_file.ino')

j = lossrun.search_rules(txt_dict,template_rules)
_image_c = cv2.imread(image_file)
for i in range(len(j)):
    cv2.circle(_image_c,(j[i][0],j[i][1]),10,(0,0,255),-1)
plt.figure(figsize=(30,25))
plt.imshow(_image_c)